In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3


engine = sqlite3.connect('example.db')
def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, engine)

In [ ]:
SET STATISTICS TIME ON

In [12]:
rsq("""EXPLAIN QUERY PLAN SELECT *
FROM Teams
-- Sub-query 1
WHERE City IN -- Sub-query filter operator
(SELECT CityName
FROM Cities) -- Table from Earthquakes database
-- Sub-query 2
AND City IN -- Sub-query filter operator
(SELECT CityName
FROM Cities
WHERE CountryCode IN ('US','CA'))
-- Sub-query 3
AND City IN -- Sub-query filter operator
(SELECT CityName
FROM Cities
WHERE Pop2017 >2000000);""")


,id,parent,notused,detail
0,2,0,0,SCAN Teams
1,7,0,0,LIST SUBQUERY 1
2,9,7,0,SCAN Cities
3,23,0,0,LIST SUBQUERY 2
4,25,23,0,SCAN Cities
5,42,0,0,LIST SUBQUERY 3
6,44,42,0,SCAN Cities


In [13]:
rsq("""-- Query 1
SELECT *
FROM Teams
-- Sub-query 1
WHERE City IN -- Sub-query filter operator
(SELECT CityName
FROM Cities) -- Table from Earthquakes database
-- Sub-query 2
AND City IN -- Sub-query filter operator
(SELECT CityName
FROM Cities
WHERE CountryCode IN ('US','CA'))
-- Sub-query 3
AND City IN -- Sub-query filter operator
(SELECT CityName
FROM Cities
WHERE Pop2017 >2000000);""")

,TeamCode,TeamName,City,State,Arena,ArenaCapacity
0,BOS,Boston Celtics,Boston,MA,TD Garden,18624
1,PHI,Philadelphia 76ers,Philadelphia,PA,Wells Fargo Center,21600
2,TOR,Toronto Raptors,Toronto,ON,Scotiabank Arena,19800
3,CHI,Chicago Bulls,Chicago,IL,United Center,20917
4,DET,Detroit Pistons,Detroit,MI,Little Caesars Arena,20491
5,ATL,Atlanta Hawks,Atlanta,GA,State Farm Arena,15711
6,MIA,Miami Heat,Miami,FL,American Airlines Arena,19600
7,DEN,Denver Nuggets,Denver,CO,Pepsi Center,19520
8,MIN,Minnesota Timberwolves,Minneapolis,MN,Target Center,19356
9,POR,Portland Trail Blazers,Portland,OR,Moda Center,19441


In [3]:
rsq("""-- Query 2
SELECT *
FROM Teams AS t
WHERE EXISTS -- Sub-query filter operator
(SELECT 1
FROM Cities AS c
WHERE t.City = c.CityName -- Columns being compared
AND c.CountryCode IN ('US','CA')
AND c.Pop2017 > 2000000);""")

,TeamCode,TeamName,City,State,Arena,ArenaCapacity
0,BOS,Boston Celtics,Boston,MA,TD Garden,18624
1,PHI,Philadelphia 76ers,Philadelphia,PA,Wells Fargo Center,21600
2,TOR,Toronto Raptors,Toronto,ON,Scotiabank Arena,19800
3,CHI,Chicago Bulls,Chicago,IL,United Center,20917
4,DET,Detroit Pistons,Detroit,MI,Little Caesars Arena,20491
5,ATL,Atlanta Hawks,Atlanta,GA,State Farm Arena,15711
6,MIA,Miami Heat,Miami,FL,American Airlines Arena,19600
7,DEN,Denver Nuggets,Denver,CO,Pepsi Center,19520
8,MIN,Minnesota Timberwolves,Minneapolis,MN,Target Center,19356
9,POR,Portland Trail Blazers,Portland,OR,Moda Center,19441


In [ ]:
SET STATISTICS TIME OFF -- Turn the time command off

In [ ]:
SET STATISTICS IO ON -- Turn the IO command on

In [7]:
rsq("""-- Example 1
SELECT CustomerID,
CompanyName,
(SELECT COUNT(*)
FROM Orders AS o -- Add table
WHERE c.CustomerID = o.CustomerID) CountOrders
FROM Customers AS c
WHERE CustomerID IN -- Add filter operator
(SELECT CustomerID
FROM Orders
WHERE ShipCity IN
('Berlin','Bern','Bruxelles','Helsinki',
'Lisboa','Madrid','Paris','London'));""")

,CustomerID,CompanyName,CountOrders
0,ALFKI,Alfreds Futterkiste,1
1,BOLID,Bolido Comidas preparadas,1
2,BSBEV,B s Beverages,7
3,CHOPS,Chop-suey Chinese,3
4,CONSH,Consolidated Holdings,2
5,EASTC,Eastern Connection,3
6,FURIB,Furia Bacalhau e Frutos do Mar,6
7,MAISD,Maison Dewey,1
8,NORTS,North-South,1
9,PRINI,Princesa Isabel Vinhos,4


In [8]:
rsq("""-- Example 2
SELECT c.CustomerID,
c.CompanyName,
COUNT(o.CustomerID)
FROM Customers AS c
INNER JOIN Orders AS o -- Join operator
ON c.CustomerID = o.CustomerID
WHERE o.ShipCity IN -- Shipping destination column
('Berlin','Bern','Bruxelles','Helsinki',
'Lisboa','Madrid','Paris','London')
GROUP BY c.CustomerID,
c.CompanyName;""")

,CustomerID,CompanyName,COUNT(o.CustomerID)
0,ALFKI,Alfreds Futterkiste,1
1,BOLID,Bolido Comidas preparadas,1
2,BSBEV,B s Beverages,7
3,CHOPS,Chop-suey Chinese,3
4,CONSH,Consolidated Holdings,2
5,EASTC,Eastern Connection,3
6,FURIB,Furia Bacalhau e Frutos do Mar,6
7,MAISD,Maison Dewey,1
8,NORTS,North-South,1
9,PRINI,Princesa Isabel Vinhos,4


In [ ]:
SET STATISTICS IO OFF -- Turn the IO command off

In [9]:
rsq("""-- Query 1
SELECT *
FROM Cities
WHERE CountryCode = 'RU' -- Country code
OR CountryCode = 'CN' -- Country code""")

,CountryCode,CityName,Pop2017,Latitude,Longitude
0,CN,Guangzhou,26940000.0,23.1300,113.2600
1,CN,Shanghai,24073000.0,31.1667,121.4667
2,CN,Beijing,18522000.0,39.9040,116.4075
3,CN,Shenzhen,17619000.0,22.5350,114.0540
4,RU,Moscow,17332000.0,55.7558,37.6178
...,...,...,...,...,...
2768,RU,Zemlya Bunge,10.0,74.8983,142.1050
2769,RU,Agapa,10.0,71.4504,89.2500
2770,RU,Tukchi,10.0,57.3670,139.5000
2771,RU,Numto,10.0,63.6667,71.3333


In [10]:
rsq("""-- Query 2
SELECT *
FROM Cities
WHERE CountryCode IN ('JM','NZ') -- Country codes""")

,CountryCode,CityName,Pop2017,Latitude,Longitude
0,NZ,Auckland,1346091.0,-36.8406,174.7400
1,JM,Kingston,580000.0,17.9714,-76.7931
2,NZ,Wellington,418500.0,-41.2889,174.7772
3,NZ,Christchurch,383200.0,-43.5310,172.6365
4,NZ,Manukau City,375600.0,-37.0000,174.8850
...,...,...,...,...,...
74,NZ,Hokitika,2892.0,-42.7156,170.9681
75,NZ,Te Anau,2250.0,-45.4167,167.7167
76,NZ,Kaikoura,2210.0,-42.4000,173.6800
77,NZ,Halfmoon Bay,300.0,-46.9000,168.1333


In [11]:
rsq("""SELECT CityName AS NearCityName,
CountryCode
FROM Cities

UNION -- Append queries

SELECT Capital AS NearCityName,
Code2 AS CountryCode
FROM Nations;""")

,NearCityName,CountryCode
0,'Ain Abessa,DZ
1,'Ain Abid,DZ
2,'Ain Arnat,DZ
3,'Ain Azel,DZ
4,'Ain Leuh,MA
...,...,...
42548,`Aynkawah,IQ
42549,`Ibri,OM
42550,`Izbat al Burj,EG
42551,`Unayzah,SA
